Wskaźniki $MACD$ i $SIGNAL$ obliczane są przy uzyciu średniej kroczącej $EMA$ dla $N$ okresów danej wzorem :

$$ EMA_N = \frac{p_0 + (1 - \alpha)p_1 + (1-\alpha)^2p_2 + ... +(1-\alpha)^N p_N}{1 + (1 - \alpha) + (1-\alpha)^2 + ... +(1-\alpha)^N}$$


gdzie $p_i$ jest próbką sprzed $i$ dni  a  $ \alpha = \frac{2}{N+1} $

Funkcja $EMA$ przyjmuje jako parametr $N$ elementowy wektor danych 

In [ ]:
def EMA(v):
    n = len(v)
    a = 2 / (n + 1)
    numerator = 0
    denominator = 0
    for i in range(n):
        numerator += v[i] * ((1 - a) ** i)
        denominator += (1 - a) ** i
    return numerator / denominator


Wskaźnik $MACD$ jest obliczany jako $EMA_{12} - EMA_{26}$ z danych, a wstaźnik $SIGNAL$ 
jako $EMA_9$ z $MACD$

Funkcja $MACD$ przyjmuje jako parametr wektor danych, którego ostatnim elementem jest dzień dla którego liczony jest wskaźnik, podobnie jak funkcja $SIGNAL$


In [ ]:
def MACD(v):
    ema_12 = EMA(v[-12:])
    ema_26 = EMA(v[-26:])
    return ema_12 - ema_26


def SIGNAL(macd):
    return EMA(macd[-9:])

Generowanie wykresu funkcji odbywa się w funkcji main, jako parametr podany jest wektor danych wczytany na początku skryptu za pomocą biblioteki pandas

In [ ]:
if __name__ == "__main__":
    column = -1
    data_dir = 'C:/Users/PCu/Downloads/data.csv'
    data = pd.read_csv(data_dir)
    data_vector = data.iloc[:, column].to_numpy()
    main(data_vector)


W funkcji main iteracyjnie wypełniane są tablice zawierające historię wartości wskaźników $MACD$ i $SIGNAL$. Zmienne ***dif*** i ***prev_dif*** zawierać będą różnicę wskaźników w obecnej i poprzedniej iteracji głównej pętli i potrzebne będą w późniejszej ocenie przydatności wskaźników w analizie technicznej.

In [ ]:
def main(data):
    #inicjalizacja zmiennych
    macd_arr = np.empty(0)
    signal_arr = np.empty(0)
    dif = 0
    prev_dif = 0

Najpierw w pętli iterującej od 26 do 35 (9 - razy - tyle ile potrzeba do wyznaczenia $SIGNAL$) elementu wektora danych obliczane są wartości $MACD$ a następnie obliczana jest pierwsza wartość $SIGNAL$

In [ ]:
     for i in range(26, 35):
        macd_arr = np.append(macd_arr, MACD(data[:i]))
        signal_arr = np.append(signal_arr, SIGNAL(macd_arr))

Przed główną pętlą programu przygotowywany jest podział wykresu na dwie części : na jednej z nich znajdować będą się wskaźniki, a na drugiej wyrysowujemy dane przy pomocy wykresy słupkowego.

In [ ]:
    fig, ax = plt.subplots(2, sharex=True)
    plt.xlabel("dni")
    ax[1].set_ylabel("cena akcji")
    ax[0].set_ylabel("wartość wskaźników")
    axes_1 = fig.gca()
    # narysowanie wykresu danych wejściowych
    ax[1].bar(np.arange(0, len(data)), data - min(data), width=1, color='#80ede2', bottom=min(data))
    # zapisujemy wartości do późniejszego użycia
    y_min_1, y_max_1 = axes_1.get_ylim()

W głównej pętli rysowanie wykresów przebiega poprzez rysowanie odcinków pomiędzy kolejnymi wartościami $MACD$ i $SIGNAL$


In [ ]:
     for i in range(35, len(data)):
            macd_arr = np.append(macd_arr,  MACD(data[:i]))
            signal_arr = np.append(signal_arr, SIGNAL(macd_arr))

            ax[0].plot([i - 1, i], [signal_arr[-2], signal_arr[-1]], color='b', label='SIGNAL')
            ax[0].plot([i - 1, i], [macd_arr[-2], macd_arr[-1]], color='r', label='MACD')
            

Jeśli wykresy $MACD$ i $SIGNAL$ przecinają się to różnica pomiędzy ich wartościami zmienia znak, w momentach przecięcia na wykresie danych rysowana jest pionowa linia - niebieska jeśli $SIGNAL$ przecina $MACD$ od góry i czerwona jeśli od dołu. Linie niebieskie wskazują moment sprzedaży akcji a czerwone zakupu.

In [ ]:
            dif = signal_arr[-1] - macd_arr[-1]

                if np.sign(dif) != np.sign(prev_dif) and i > 35:
            
                    if dif - prev_dif < 0:
                        ax[1].vlines(i, y_min_1, y_max_1, colors='b')
                    else:
                        ax[1].vlines(i, y_min_1, y_max_1, colors='r')

            prev_dif = dif

Po wykonaniu się głownej pęli dodawana jest legenda do wykresów i są one wyświetlane.

In [ ]:
    ax[0].legend(['SIGNAL', 'MACD'])
    ax[1].legend(['buy', 'sell'], loc='lower left')
    fig.tight_layout()
    plt.show()

Poniżej załączony jest wynik działania programu dla danych wejściowych w postaci wartości WIG20 dla 1000 dni

![Wykres dla 1000 okresów](plot_1000.png)

Do oceny przydatności wskaźnika w analizie technicznej bardziej przydatne mogą być wykresy na mniejszych danych (ze względu na czytelność). Poniżej prezentowany jest wykres dla pierwszych 100 okresów danych wejściowych.

![Wykres dla 100 okresów](plot100.png)

Na podstawie wykresu można stwierdzić, że rozpatrywane wskaźniki w przybliżeniu dobrze przewidują wzrost / spadek ceny akcji, ale moment kupna / sprzedaży wskazany przez przecięcie się wykresów jest opóźniony, przez co sugerowanie się tymi punktami w handlu akcjami (przy założeniu że cena nie rośnie stale, a jest ciągle zmienna) nie przyniesie korzyści.